In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

#import lightgbm as lgb
import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
# Low MI scores
for pp in ["cat2","cat3","cat4","cat6"]:
    train.pop(pp)
    test.pop(pp)
train.head()

,id,cat0,cat1,cat5,cat7,cat8,cat9,cont0,cont1,cont2,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,E,C,N,0.201470,-0.014822,0.669699,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,D,F,A,O,0.743068,0.367411,1.021605,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,D,D,A,F,0.742708,0.310383,-0.012673,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,D,E,C,K,0.429551,0.620998,0.577942,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,D,E,A,N,1.058291,0.367492,-0.052389,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [6]:
# Separate target from features
y = train['target']
features = train.drop(['id','target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat5,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,B,B,D,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,A,A,D,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,B,B,D,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,A,A,D,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


# Preprocessing

In [7]:

# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()


,cat0,cat1,cat5,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,1.0,1.0,1.0,4.0,2.0,13.0,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,1.0,1.0,3.0,5.0,0.0,14.0,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,0.0,0.0,3.0,3.0,0.0,5.0,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,1.0,1.0,3.0,4.0,2.0,10.0,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,0.0,0.0,3.0,4.0,0.0,13.0,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


In [8]:
'''
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# one-hot-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()

oh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_ohe = oh_encoder.fit_transform(features[object_cols])
X_test_ohe = oh_encoder.transform(test[object_cols])

X_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])
X_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])

X = pd.concat([X, X_ohe], axis=1)
X_test = pd.concat([X_test, X_test_ohe], axis=1)
X = X.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)
    
# Preview the one-hot-encoded features
X.head()
'''

'\n# List of categorical columns\nobject_cols = [col for col in features.columns if \'cat\' in col]\n\n# one-hot-encode categorical columns\nX = features.copy()\nX_test = test.drop([\'id\'], axis=1).copy()\n\noh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")\nX_ohe = oh_encoder.fit_transform(features[object_cols])\nX_test_ohe = oh_encoder.transform(test[object_cols])\n\nX_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])\nX_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])\n\nX = pd.concat([X, X_ohe], axis=1)\nX_test = pd.concat([X_test, X_test_ohe], axis=1)\nX = X.drop(object_cols, axis=1)\nX_test = X_test.drop(object_cols, axis=1)\n    \n# Preview the one-hot-encoded features\nX.head()\n'

In [9]:
scale_features = [col for col in features.columns if 'cont' in col]

ss = StandardScaler()
X[scale_features] = ss.fit_transform(features[scale_features])
X_test[scale_features] = ss.transform(test[scale_features])

In [10]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Optuna

In [11]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [12]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=X, y=y):
  """
  """
  param_space = {
               'objective': 'reg:squarederror',
               'learning_rate':trial.suggest_uniform('learning_rate', 1e-4, 4.5e-2),
               'n_estimators':N_ESTIMATORS,
               'subsample': trial.suggest_uniform('subsample', 0.697, 0.897),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.006, 0.206),
              'max_depth':trial.suggest_int('max_depth', 2, 8),
              'lambda':trial.suggest_uniform('lambda', 13.4, 25.4),
              'alpha':trial.suggest_uniform('alpha', 4.2, 16.2),
           'min_child_weight':trial.suggest_uniform('min_child_weight', 305, 415),
           'importance_type': 'total_gain',
            'tree_method': 'hist'
                }
            
  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  #lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  pred=np.zeros(y_va.shape[0])
    
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
    
  #X_tr = apply_noise_row(X_tr)

  
  #start = time.time()
  for inseed in seed_list:
    param_space['seed'] = inseed

    model = xgb.XGBRegressor(**param_space)
    model.fit(
        X_tr, 
        y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE
    )


    pred += model.predict(X_va) / len(seed_list)
    
  #elapsed = time.time() - start
  rmse = mean_squared_error(y_va, pred, squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [13]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-26 08:11:23,429] A new study created in memory with name: no-name-10688b46-c0ec-49d8-bbba-d2de1a1f5626


[0]	validation_0-rmse:7.65988
[1000]	validation_0-rmse:0.72421
[2000]	validation_0-rmse:0.71849
[3000]	validation_0-rmse:0.71671
[4000]	validation_0-rmse:0.71632
[4112]	validation_0-rmse:0.71630
[0]	validation_0-rmse:7.65988
[1000]	validation_0-rmse:0.72426
[2000]	validation_0-rmse:0.71825
[3000]	validation_0-rmse:0.71659
[4000]	validation_0-rmse:0.71623
[4242]	validation_0-rmse:0.71625


[I 2021-08-26 08:14:56,874] Trial 0 finished with value: 0.7162477946192615 and parameters: {'learning_rate': 0.015054316946396156, 'subsample': 0.8542964173789822, 'colsample_bytree': 0.07112232250344032, 'max_depth': 8, 'lambda': 14.518385867574269, 'alpha': 5.145921422660868, 'min_child_weight': 380.97133921828686}. Best is trial 0 with value: 0.7162477946192615.


[0]	validation_0-rmse:7.64962
[1000]	validation_0-rmse:0.72268
[2000]	validation_0-rmse:0.71830
[3000]	validation_0-rmse:0.71654
[4000]	validation_0-rmse:0.71566
[5000]	validation_0-rmse:0.71518
[6000]	validation_0-rmse:0.71496
[7000]	validation_0-rmse:0.71481
[7622]	validation_0-rmse:0.71479
[0]	validation_0-rmse:7.64962
[1000]	validation_0-rmse:0.72274
[2000]	validation_0-rmse:0.71826
[3000]	validation_0-rmse:0.71650
[4000]	validation_0-rmse:0.71563
[5000]	validation_0-rmse:0.71522
[6000]	validation_0-rmse:0.71502
[7000]	validation_0-rmse:0.71491
[7041]	validation_0-rmse:0.71490


[I 2021-08-26 08:19:58,608] Trial 1 finished with value: 0.7147838717852049 and parameters: {'learning_rate': 0.01638834904891358, 'subsample': 0.7337548369555916, 'colsample_bytree': 0.15774873412175436, 'max_depth': 4, 'lambda': 24.19584127209603, 'alpha': 4.789725531986864, 'min_child_weight': 388.55140950800023}. Best is trial 1 with value: 0.7147838717852049.


...

[0]	validation_0-rmse:7.47890
[1000]	validation_0-rmse:0.72496
[2000]	validation_0-rmse:0.72125
[3000]	validation_0-rmse:0.71908
[4000]	validation_0-rmse:0.71765
[5000]	validation_0-rmse:0.71678
[6000]	validation_0-rmse:0.71613
[7000]	validation_0-rmse:0.71565
[8000]	validation_0-rmse:0.71537
[9000]	validation_0-rmse:0.71519
[10000]	validation_0-rmse:0.71503
[11000]	validation_0-rmse:0.71496
[11494]	validation_0-rmse:0.71494
[0]	validation_0-rmse:7.47889
[1000]	validation_0-rmse:0.72517
[2000]	validation_0-rmse:0.72114
[3000]	validation_0-rmse:0.71899
[4000]	validation_0-rmse:0.71760
[5000]	validation_0-rmse:0.71672
[6000]	validation_0-rmse:0.71609
[7000]	validation_0-rmse:0.71566
[8000]	validation_0-rmse:0.71537
[9000]	validation_0-rmse:0.71517
[10000]	validation_0-rmse:0.71504
[11000]	validation_0-rmse:0.71496
[11272]	validation_0-rmse:0.71496


[I 2021-08-26 10:38:30,054] Trial 28 finished with value: 0.7149057320481842 and parameters: {'learning_rate': 0.03855176871987986, 'subsample': 0.738993188751685, 'colsample_bytree': 0.10841792917976853, 'max_depth': 2, 'lambda': 19.593482368069125, 'alpha': 8.042277631077042, 'min_child_weight': 379.06054252460507}. Best is trial 15 with value: 0.7146166263980875.


[0]	validation_0-rmse:7.50153
[1000]	validation_0-rmse:0.71549
[1366]	validation_0-rmse:0.71546
[0]	validation_0-rmse:7.50154
[1000]	validation_0-rmse:0.71550
[1385]	validation_0-rmse:0.71548


[I 2021-08-26 10:39:51,047] Trial 29 finished with value: 0.7152189986236754 and parameters: {'learning_rate': 0.03560812312133998, 'subsample': 0.79624276741487, 'colsample_bytree': 0.17820560725513004, 'max_depth': 8, 'lambda': 17.014520904901932, 'alpha': 10.720326474296842, 'min_child_weight': 414.82226759044715}. Best is trial 15 with value: 0.7146166263980875.


Number of finished trials: 30
Best trial: {'learning_rate': 0.036955320792692224, 'subsample': 0.7635473153188327, 'colsample_bytree': 0.1354938997791723, 'max_depth': 3, 'lambda': 22.821468405823435, 'alpha': 9.002866837758342, 'min_child_weight': 402.0811006541652}


In [14]:
study.best_params

{'learning_rate': 0.036955320792692224,
 'subsample': 0.7635473153188327,
 'colsample_bytree': 0.1354938997791723,
 'max_depth': 3,
 'lambda': 22.821468405823435,
 'alpha': 9.002866837758342,
 'min_child_weight': 402.0811006541652}

# Log

====== Ordinal encoding =========

0.7150466204380306 no noise ver1

0.7146430566586621 no noise ver3 (narrow space)

====== One-hot encoding =========

0.7153005695083217 no noise ver2
